# mun_gdf 2015 vs mun_gdf 2020 CVEGEOs

This notebook finds the differences in CVEGEOs between 2015 and 2020 metropolis gdfs in order to find which cities have new CVEGEOs (And therefore, which cities we do not have 2020 OSMnx network fully downloaded)

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/jovyan/accesibilidad-urbana/aup/data.py:25: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


## Load data

In [2]:
metro_schema = 'metropolis'
metro_table = 'metro_gdf_2015'

metro_2015 = aup.gdf_from_db(metro_table, metro_schema)

# Show
print(metro_2015.shape)
metro_2015.head(1)

(414, 6)


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry,city
0,01001,01,001,Aguascalientes,"POLYGON ((-102.10641 22.06035, -102.10368 22.0...",Aguascalientes


In [3]:
metro_schema = 'metropolis'
metro_table = 'metro_gdf_2020'

metro_2020 = aup.gdf_from_db(metro_table, metro_schema)

# Show
print(metro_2020.shape)
metro_2020.head(1)

(367, 6)


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry,city
0,01001,01,001,Aguascalientes,"POLYGON ((-102.10641 22.06035, -102.10368 22.0...",Aguascalientes


## Prepare data comparison

In [350]:
summary_2015 = pd.DataFrame()
summary_2020 = pd.DataFrame()
processed_city_list = []
i = 0

#----------------------------------------------- Process cities from metro_2015
for city in list(metro_2015.city.unique()):

    # Isolate city data
    city_gdf = metro_2015.loc[metro_2015.city == city]
    cvegeos =  (list(city_gdf.CVEGEO.unique()))
    
    # Add city data to summary df
    row_data = [city, cvegeos, len(cvegeos)]
    summary_2015.loc[i,('city','cvegeos_15','quantity_15')] = row_data
     	 	
    # Added new city (row)
    i = i+1
    
    processed_city_list.append(city)

#----------------------------------------------- Process cities from metro_2020
for city in list(metro_2020.city.unique()):

    # Isolate city data
    city_gdf = metro_2020.loc[metro_2020.city == city]
    cvegeos =  (list(city_gdf.CVEGEO.unique()))
    
    # Add city data to summary df
    row_data = [city, cvegeos, len(cvegeos)]
    summary_2020.loc[i,('city','cvegeos_20','quantity_20')] = row_data # Added this way because it wouldn't let me add a list to a DataFrame cell.
     	 	
    # Added new city (row)
    i = i+1
    
    processed_city_list.append(city)

/opt/conda/envs/gds/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:950: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)
/opt/conda/envs/gds/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
/opt/conda/envs/gds/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is d

## Compare data

In [351]:
#----------------------------------------------- Merge 2015 and 2020 summary dfs
# Merge
summary = pd.merge(summary_2015,summary_2020, on='city',how='outer')

# Remove nans
idx = summary['cvegeos_15'].isna()
summary.loc[idx,'cvegeos_15'] = '0' #Iterable in loop that finds changes in cvegeos one by one

idx = summary['quantity_15'].isna()
summary.loc[idx,'quantity_15'] = 0

idx = summary['cvegeos_20'].isna()
summary.loc[idx,'cvegeos_20'] = '0' #Iterable in loop that finds changes in cvegeos one by one

idx = summary['quantity_20'].isna()
summary.loc[idx,'quantity_20'] = 0

#----------------------------------------------- Evaluate result
# CVEGEOs quantity diff (2020-2015) by city
summary['quantity_diff'] = summary['quantity_20'] - summary['quantity_15']

# CVEGEOs difference nature
summary = summary.set_index('city')
for city in processed_city_list:
    
    city_cvegeos_2015 = summary.loc[city,'cvegeos_15']
    city_cvegeos_2020 = summary.loc[city,'cvegeos_20']
    
    # Find changes in cvegeos
    summary.loc[city,'lost_CVEGEOs'] = False
    for cvegeo in city_cvegeos_2015:
        if cvegeo not in city_cvegeos_2020:
            summary.loc[city,'lost_CVEGEOs'] = True

    summary.loc[city,'new_CVEGEOs'] = False
    for cvegeo in city_cvegeos_2020:
        if cvegeo == '0':
            summary.loc[city,'new_CVEGEOs'] = 'n/a'
            pass
        elif cvegeo not in city_cvegeos_2015:
            summary.loc[city,'new_CVEGEOs'] = True

# Show
update_problem = summary.loc[summary.new_CVEGEOs == True]
update_problem.drop('CDMX',inplace=True)
update_problem

/tmp/ipykernel_17938/515463071.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  update_problem.drop('CDMX',inplace=True)


,cvegeos_15,quantity_15,cvegeos_20,quantity_20,quantity_diff,lost_CVEGEOs,new_CVEGEOs
city,,,,,,,
Tapachula,[07089],1.0,"[07089, 07055, 07102]",3.0,2.0,False,True
Delicias,"[08021, 08045]",2.0,"[08021, 08045, 08055]",3.0,1.0,False,True
ZMVM,"[09002, 09003, 09004, 09005, 09006, 09007, 090...",75.0,"[13013, 13069, 15002, 15011, 15013, 15020, 150...",47.0,-28.0,True,True
Leon,"[11020, 11037]",2.0,"[11020, 11025, 11031]",3.0,1.0,True,True
Cuernavaca,"[17007, 17008, 17009, 17011, 17018, 17020, 170...",8.0,"[17007, 17008, 17009, 17011, 17018, 17020, 170...",10.0,2.0,True,True
Oaxaca,"[20045, 20063, 20067, 20083, 20087, 20091, 201...",24.0,"[20023, 20045, 20063, 20067, 20083, 20087, 200...",26.0,2.0,False,True
Puebla,"[21001, 21015, 21034, 21041, 21048, 21060, 210...",37.0,"[21001, 21015, 21034, 21041, 21090, 21106, 211...",28.0,-9.0,True,True
Tlaxcala,"[29001, 29002, 29005, 29009, 29010, 29018, 290...",19.0,"[29001, 29002, 29003, 29005, 29009, 29010, 290...",24.0,5.0,False,True
Cordoba,"[30014, 30044, 30068, 30196]",4.0,"[30014, 30044, 30068, 30113, 30196]",5.0,1.0,False,True


### Método anterior (Equivocado)

In [346]:
summary = pd.DataFrame()
processed_city_list = []
i = 0

#----------------------------------------------- Process cities from metro_2015
for city in list(metro_2015.city.unique()):

    # Isolate city data
    city_gdf = metro_2015.loc[metro_2015.city == city]
    cvegeos =  (list(city_gdf.CVEGEO.unique()))
    
    # Add city data to summary df
    summary.loc[i,'city'] = city
    summary.loc[i] = str(cvegeos)
    summary.loc[i,'quantity_15'] = len(cvegeos)
     	 	
    # Added new city (row)
    i = i+1
    
    processed_city_list.append(city)

#----------------------------------------------- Process cities from metro_2020
for city in list(metro_2020.city.unique()):
    
    # Isolate city data
    city_gdf = metro_2020.loc[metro_2020.city == city]
    cvegeos = list(city_gdf.CVEGEO.unique())
    
    # If it was already added, add new column to city (idx)
    if city in processed_city_list:
        idx = summary['city'] == city
        
        summary.loc[idx,'cvegeos_20'] = str(cvegeos)
        summary.loc[idx,'quantity_20'] = len(cvegeos)
        
    # Else, add new city row                                
    else:
        summary.loc[i,'city'] = city
        # 2015 data does not exist for this city
        summary.loc[i,'cvegeos_15'] = 'No esta en metro_2015'
        summary.loc[i,'quantity_15'] = 0
        # Add 2020 data
        summary.loc[i,'cvegeos_20'] = str(cvegeos)
        summary.loc[i,'quantity_20'] = len(cvegeos)
        # Added new city (row)
        i = i+1
        
    processed_city_list.append(city)

#----------------------------------------------- Evaluate result
# CVEGEOs quantity diff (2020-2015) by city
summary['quantity_diff'] = summary['quantity_20'] - summary['quantity_15']


summary = summary.set_index('city')
for city in processed_city_list:
    city_cvegeos_2015 = summary.loc[city,'cvegeos_15']
    city_cvegeos_2020 = summary.loc[city,'cvegeos_20']
    quantity_check = summary.loc[city,'quantity_diff']

    # If city's 2015 CVEGEOs and 2020 CVEGEOs are different, BUT quantity difference is 0, then city's CVEGEOs were modified but ended up with the same amount.
    # (Dif en cvegeos --> Update problem)
    if (city_cvegeos_2015 != city_cvegeos_2020) & (quantity_check == 0.0):
        diff_result = 'Dif en cvegeos'
        summary.loc[city,'cvegeos_diff'] = diff_result

    # If city's 2015 CVEGEOs and 2020 CVEGEOs are different, AND quantity difference is not 0, then city's CVEGEOs list is different because new were added/removed.
    elif (city_cvegeos_2015 != city_cvegeos_2020) & (quantity_check != 0.0):
        diff_result = 'Solo diff por quantity'
        summary.loc[city,'cvegeos_diff'] = diff_result
    else:
        diff_result = 'No cvegeo diff'
        summary.loc[city,'cvegeos_diff'] = diff_result

    summary['use'] = summary['quantity_20'].isna()
    use = summary.loc[city,'use']
    
    quantity_diff = summary.loc[city,'quantity_diff']
    if use == True:
        summary.loc[city,'Result'] = 'N/A'
    elif (quantity_diff > 0) or (diff_result == 'Dif en cvegeos'):
        summary.loc[city,'Result'] = 'Update problem'
    else:
        summary.loc[city,'Result'] = 'No problem'

    summary.drop(columns='use',inplace=True)
        

print(f'{summary["quantity_15"].sum()} of {len(metro_2015)}.')
print(f'{summary["quantity_20"].sum()} of {len(metro_2020)}.')

KeyError: 'Aguascalientes'